In [ ]:
import json
import os
from os.path import join as pjoin
import plotly.plotly as py
import plotly.graph_objs as go

DATA_DIR = './data'

modules_performance = {}

# Parse the input JSON files
for filename in os.listdir(DATA_DIR):
    filename = pjoin(DATA_DIR, filename)
    with open(filename) as data_file:
        data_string = data_file.read()
        try:
            df = json.loads(data_string)
            module_name = df['Probes'][0]['Name']

            if module_name not in modules_performance:
                modules_performance[module_name] = {}

            probes_mean_time = df['Probes'][0]['Mean']
            config_date = df['ITK_MANUAL_BUILD_INFO']['GIT_CONFIG_DATE']
            probes = config_date, probes_mean_time

            itk_version = df['SystemInformation']['ITKVersion']

            if itk_version in modules_performance[module_name]:
                modules_performance[module_name][itk_version].append(probes)
            else:
                modules_performance[module_name][itk_version] = []
                modules_performance[module_name][itk_version].append(probes)

        except ValueError:
            print(repr(data_string))

modules_worst_performance = {}

# Reduce the data: pick the slowest mean probes time for each module
for module_name, module_dict in modules_performance.items():
    for itk_version, probes in module_dict.items():
        probes_worst_mean_time = min(probes, key=lambda x:x[1])

        if module_name not in modules_worst_performance:
                modules_worst_performance[module_name] = {}

        probes = (probes, key=lambda x:x[0]), probes_worst_mean_time
        modules_worst_performance[module_name][itk_version] = []
        modules_worst_performance[module_name][itk_version].append(probes)

# Prepare the data: if a given module was not present in a given version, add
# an empty performance value

# Get the modules and the versions as lists

module_names = []
itk_versions = []
for module_name, module_dict in modules_worst_performance.items():
        if module_name not in module_names:
                module_names.append(module_name)

        for itk_version, probes in module_dict.items():
                if itk_version not in itk_versions:
                        itk_versions.append(itk_version)

# Add empty performance values for non-existing data
for module_name, itk_version in zip(module_names, itk_version):
        if itk_version not in modules_worst_performance[module_name]:
                probes = (probes, key=lambda x:x[0]), 0
                modules_worst_performance[module_name][itk_version].append(probes)


# Heatmap plot
z = []

for module_name, module_dict in modules_worst_performance.items():
    new_row = []
    for itk_version, probes in module_dict.items():
        new_row.append( probes[1] )
    z.append(list(new_row))

data = [
    go.Heatmap(
        z=z,
        x=date_list,
        y=programmers,
        colorscale='Viridis',
    )
]

layout = dict(title='ITK Performance Stats: {} <br>Worst Performance',
              xaxis=dict(title='ITK version', ticks='', nticks=36),
              yaxis=dict(title='Module Name', ticks=''),
              showlegend=True
              )

py.sign_in('name', 'API_key')

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)